# 使用阶跃星辰 API 进行互联网搜索指南

本 `ipynb` 文件旨在演示如何利用阶跃星辰（StepFun）大模型 API 的互联网搜索功能。通过该功能，大模型可以获取实时信息，从而提高回答的质量、时效性和准确性。

**核心功能**：通过在 API 请求中加入特定的 `tools` 参数来启用 `web_search` 工具。

## 1. 启用互联网搜索

要启用互联网搜索功能，你需要在调用 API 时传入 `tools` 参数。该参数是一个列表，其中包含一个定义好的 `web_search` 工具对象。

- `type`: 固定为 `"web_search"`。
- `function.description`: 用来描述何时应使用此工具，这会引导大模型在需要时自动调用互联网搜索。

In [1]:
# 构建 web_search 工具的定义
tools = [
    {
        "type": "web_search",  # 类型是固定的
        "function": {
            "description": "这个 web_search 用来搜索互联网上的实时信息。"  # 描述什么情况下需要搜索
        }
    }
]

print("Web search tool is defined:", tools)

Web search tool is defined: [{'type': 'web_search', 'function': {'description': '这个 web_search 用来搜索互联网上的实时信息。'}}]


## 2. 实现步骤 & 参考代码

下面我们将演示如何在非流式和流式响应中实现联网搜索。

### 2.1 非流式响应

在非流式模式下，API 会一次性返回所有结果，包括模型最终的回答和搜索过程的详细信息。

In [5]:
import os
from openai import OpenAI

# 建议从环境变量或安全存储中读取 API Key
STEPFUN_API_KEY = os.getenv("STEPFUN_API_KEY")

client = OpenAI(
    base_url="https://api.stepfun.com/v1",
    api_key=STEPFUN_API_KEY
)

# 1. 定义 System Prompt 和 User Prompt
sys_prompt = "请先使用 web_search 工具搜索互联网，然后根据搜索结果来回答用户的问题。注意：1.在接到问题后，【先搜索后回答】。 2.在接到用户问题后,【立刻将用户的问题总结为搜索关键字】，【调用 websearch 工具】搜更多信息,之后根据上下文回答用户问题。"
# user_prompt = "世界上最大的湖泊在哪里"
user_prompt= "今天几号，星期几"


# 2. 构建消息体
messages = [
    {"role": "system", "content": sys_prompt},
    {"role": "user", "content": user_prompt}
]

# 3. 构建 web_search 工具
tools = [
    {
        "type": "web_search",
        "function": {
            "description": "用于搜索互联网以获取最新信息。"
        }
    }
]

# 4. 调用 API
try:
    completion = client.chat.completions.create(
        model="step-2-mini",
        messages=messages,
        tools=tools,
        # tool_choice="auto",  # 让模型自动决定是否使用工具
        stream=False
    )
    
    # 5. 打印完整返回结果
    print("------ 模型最终回答 ------")
    print(completion.choices[0].message.content)
    print("\n------ 搜索工具调用详情 ------")
    print(completion.choices[0].message.tool_calls)

except Exception as e:
    print(f"API 调用失败: {e}")

------ 模型最终回答 ------
今天是2025年7月10日，星期四。

------ 搜索工具调用详情 ------
[ChatCompletionMessageFunctionToolCall(id='call_3kqwm1LRSpObB274S1hYUQ', function=Function(arguments='{"keyword": "today date"}', name='step_websearch', results=[{'index': 0, 'url': 'https://wk.baidu.com/view/f26576e04631b90d6c85ec3a87c24028905f8520', 'title': '今天的英文怎么写', 'summary': "今天的日期 英文：Today's date 词语解释： today 英 [təˈdeɪ] 美 [təˈde] adv. 现在；现今；（在）今天；现代，当代 n. 现今；今天，今日；现代，当代 例句： 1、How are you feeling today? 你今天感觉怎么样？ 2、The United States is in a serious recession today 当今美国正处于严重的衰退时期。 date 英 [deɪt] 美 [det] n. 约会；日期，日子；时代，年代；（北非和西亚常见的海枣树的）海枣 vt. 过时；使…显老；显示出…时代（或年龄）；鉴定…的年代 vt.& vi. 与人约会，相约 vi. 属于某一特定时期；从某时起计算日期；记载日期；注有（或注明）日期 例句： What's the date today? 今天是几号？2.今天的日期用英语怎么写"}, {'index': 1, 'url': 'https://wenwen.sogou.com/z/q799810436.htm', 'title': '今天的日期用英语怎么写？？？', 'summary': "今天的日期英文：Today's date词语解释：today英 [təˈdeɪ]   美 [təˈde]adv.  现在;现今;（在）今天;现代，当代n.  现今;今天，今日;现代，当代例句：1、How are you feeling today?你今天感觉怎么样？2、The United St

### 2.2 流式响应

在流式模式下，结果会以数据块（chunk）的形式逐步返回。对于搜索功能，第一个 chunk 中会包含完整的搜索结果 `tool_calls`，后续的 chunk 则会逐步返回模型的生成内容。

这种方式可以让你更快地向用户展示搜索到的信息（例如渲染引用的链接），同时等待模型生成最终答案。

In [6]:
import os
from openai import OpenAI

# 建议从环境变量或安全存储中读取 API Key
STEPFUN_API_KEY = os.getenv("STEPFUN_API_KEY") 

client = OpenAI(
    base_url="https://api.stepfun.com/v1",
    api_key=STEPFUN_API_KEY
)

messages = [
    {"role": "system", "content": "请先搜索，然后根据搜索结果回答。"},
    {"role": "user", "content": "今天日期"}#2025年巴黎奥运会有哪些新增项目？
]

tools = [
    {
        "type": "web_search",
        "function": {
            "description": "用于搜索互联网以获取体育赛事等最新信息。"
        }
    }
]

try:
    stream = client.chat.completions.create(
        model="step-1-8k",
        messages=messages,
        tools=tools,
        tool_choice="auto",
        stream=True
    )

    print("------ 流式响应开始 ------\n")
    full_content = ""
    # 首次迭代标记
    is_first_chunk = True
    
    for chunk in stream:
        # 搜索引擎的结果会在第一个 chunk 中返回
        if is_first_chunk and chunk.choices[0].delta.tool_calls:
            print("--- 搜索结果（在第一个 chunk 中返回） ---")
            tool_calls = chunk.choices[0].delta.tool_calls
            for tool_call in tool_calls:
                if tool_call.function.results:
                    # 将字符串结果解析为 Python 对象
                    import json
                    search_results = json.loads(tool_call.function.results)
                    print(f"搜索关键词: {tool_call.function.arguments}")
                    print("搜索到的链接：")
                    for result in search_results:
                        print(f"  - [{result['title']}]({result['url']})")
            print("-------------------------------------\n")
            is_first_chunk = False

        # 打印模型生成的文本内容
        content = chunk.choices[0].delta.content
        if content:
            print(content, end='', flush=True)
            full_content += content

    print("\n\n------ 流式响应结束 ------")

except Exception as e:
    print(f"API 调用失败: {e}")

------ 流式响应开始 ------

对不起，我无法获取实时日期信息。但你可以查看你的设备或访问互联网来获取当前日期。如果你需要帮助，请告诉我。

------ 流式响应结束 ------


## 3. 返回结果解析

调用 API 后，返回结果中的 `message.tool_calls` 字段包含了搜索的详细信息。其中 `results` 字段是搜索引擎返回的原始信息，你可以基于此来渲染 UI，例如显示引用的来源链接。

下面是一个非流式响应的返回结果示例：

In [ ]:
response_example = {
  "id": "354dff9f4e8b0d273cf9d4639ba5f183.2618f2a05bbfdd02896c32051994da8c",
  "object": "chat.completion",
  "created": 1722413870,
  "model": "step-1-8k",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "上海中心大厦",
        "tool_calls": [
          {
            "id": "call__Tg9p9DHRK2JamqimG-IVA",
            "type": "web_search",
            "function": {
              "name": "step_websearch",
              "arguments": "{\"keyword\": \"上海最高的楼\"}",
              "results": "[...]"  // 这里是包含多个搜索结果（url, title, summary）的JSON字符串
            }
          }
        ]
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 1516,
    "completion_tokens": 4,
    "total_tokens": 1520
  }
}

import json
print(json.dumps(response_example, indent=2, ensure_ascii=False))

## 4. 常见用法和注意事项

1.  **清晰的指令**: 为了让模型准确判断何时调用搜索，你需要提供清晰的 `System Prompt` 和 `tool.function.description`。

2.  **输出引用链接**: 你可以明确要求模型在回答时附上参考链接。通过解析返回结果中的 `tool_calls` -> `function` -> `results` 字段，可以获取到搜索到的 URL、标题和摘要，并将其展示给用户。

3.  **流式输出的特点**: 在使用流式输出时，所有 `tool_calls` 的信息（包括搜索结果）会**在第一个返回的 chunk 中一次性提供**。你可以利用这一点来优先渲染 UI 元素（如“正在搜索”、“参考链接”等）。

4.  **意图判断**: 模型会先进行意图判断，评估用户的提问是否真的需要联网搜索。并非每次调用都会触发搜索，这有助于节省成本和提高效率。

5.  **计费**: 互联网搜索工具按**实际调用次数**计费，具体请参考官方的定价说明。